In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch
import os,sys
import torchvision.transforms as transforms
# os.environ["https_proxy"]="10.10.20.100:1089"
# os.environ["https_proxy"]="127.0.0.1:7890"
os.environ["CUDA_VISIBLE_DEVICES"]="7"
sys.path.append('..')
sys.path.append('.')
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
from pathlib import Path
from loguru import logger
from datetime import datetime

from hyvideo.utils.file_utils import save_videos_grid
from hyvideo.config import parse_args
from hyvideo.inference import HunyuanVideoSampler

/mnt/public/hanling/mnt/public/hanling/miniconda3/envs/HunyuanVideo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
import hyvideo.config
print(hyvideo.config.__file__)
from hyvideo.config import parse_args
string_args="""--video-size 720 1280 --video-length 129 --infer-steps 50 --prompt cat. --flow-reverse --use-cpu-offload --save-path ./results"""
string_args=string_args.split(" ")
print(string_args)
args = parse_args(string_args=string_args)
print(args)
prompt1="A man waits at a bus stop; a bus arrives."
prompt11="Cartoon style. A man stands at a quiet bus stop on a sunny afternoon. Then, a bright yellow bus approaches."
prompt12="Realistic style. A man stands at a quiet bus stop on a sunny afternoon. Then, a bright yellow bus approaches."
prompt2="A man is reading the newspaper on the chair, then he walks away."
args.prompt=prompt1


/mnt/public/wangsiyuan/HunyuanVideo_efficiency/hyvideo/config.py
['--video-size', '720', '1280', '--video-length', '129', '--infer-steps', '50', '--prompt', 'cat.', '--flow-reverse', '--use-cpu-offload', '--save-path', './results']
Namespace(model='HYVideo-T/2-cfgdistill', latent_channels=16, precision='bf16', rope_theta=256, vae='884-16c-hy', vae_precision='fp16', vae_tiling=True, text_encoder='llm', text_encoder_precision='fp16', text_states_dim=4096, text_len=256, tokenizer='llm', prompt_template='dit-llm-encode', prompt_template_video='dit-llm-encode-video', hidden_state_skip_layer=2, apply_final_norm=False, text_encoder_2='clipL', text_encoder_precision_2='fp16', text_states_dim_2=768, tokenizer_2='clipL', text_len_2=77, denoise_type='flow', flow_shift=7.0, flow_reverse=True, flow_solver='euler', use_linear_quadratic_schedule=False, linear_schedule_end=25, model_base='ckpts', dit_weight='ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states.pt', model_resolution='540p'

In [4]:
models_root_path = Path(args.model_base)
hunyuan_video_sampler = HunyuanVideoSampler.from_pretrained(models_root_path, args=args)

2025-01-31 14:24:02.551 | INFO     | hyvideo.inference:from_pretrained:154 - Got text-to-video model root path: ckpts
2025-01-31 14:24:02.552 | INFO     | hyvideo.inference:from_pretrained:189 - Building model...
2025-01-31 14:24:02.799 | INFO     | hyvideo.inference:load_state_dict:340 - Loading torch model ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states.pt...
/mnt/public/wangsiyuan/HunyuanVideo_efficiency/hyvideo/inference.py:341: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to

In [25]:
from hyvideo.vae.adaptive_temporal_tiling import AdaptiveTemporalTiling
device = "cuda" if torch.cuda.is_available() else "cpu"

adaptor = AdaptiveTemporalTiling(
        vae_ckpt_path="ckpts/hunyuan-video-t2v-720p/vae",
        device=device,
        vae_precision="fp16",
    )



2025-01-31 15:07:12.678 | INFO     | hyvideo.vae.adaptive_temporal_tiling:__init__:49 - Loading 4x-time-compression VAE ...
2025-01-31 15:07:15.786 | INFO     | hyvideo.vae.unet_causal_3d_blocks:apply_t_ops_config:742 - Down Conv Stride updated to (2, 2, 2)
2025-01-31 15:07:15.788 | INFO     | hyvideo.vae.unet_causal_3d_blocks:apply_t_ops_config:742 - Down Conv Stride updated to (4, 2, 2)
2025-01-31 15:07:15.789 | INFO     | hyvideo.vae.unet_causal_3d_blocks:apply_t_ops_config:742 - Down Conv Stride updated to (2, 2, 2)
2025-01-31 15:07:15.881 | INFO     | hyvideo.vae.adaptive_temporal_tiling:__init__:59 - Loading 2x-time-compression VAE ...
2025-01-31 15:07:18.821 | INFO     | hyvideo.vae.unet_causal_3d_blocks:apply_t_ops_config:742 - Down Conv Stride updated to (2, 2, 2)
2025-01-31 15:07:18.825 | INFO     | hyvideo.vae.unet_causal_3d_blocks:apply_t_ops_config:742 - Down Conv Stride updated to (2, 2, 2)
2025-01-31 15:07:18.826 | INFO     | hyvideo.vae.unet_causal_3d_blocks:apply_t_ops

In [29]:
hunyuan_video_sampler.vae.adaptor=adaptor
args.infer_steps=50
args.video_length=128*2+1
height=720//4
width=1280//4
for seed in range(10):
    outputs = hunyuan_video_sampler.predict(
            prompt=prompt11, 
            height=height,
            width=width,
            video_length=args.video_length,
            seed=seed,
            negative_prompt=args.neg_prompt,
            infer_steps=args.infer_steps,
            guidance_scale=args.cfg_scale,
            num_videos_per_prompt=args.num_videos,
            flow_shift=args.flow_shift,
            batch_size=args.batch_size,
            embedded_guidance_scale=args.embedded_cfg_scale
        )
    samples = outputs['samples']
    save_path = f"output/raw_prompt1_{seed}.mp4"
    save_videos_grid(samples, save_path, fps=24)

2025-01-31 15:15:22.587 | INFO     | hyvideo.inference:predict:580 - Input (height, width, video_length) = (180, 320, 257)
2025-01-31 15:15:22.600 | DEBUG    | hyvideo.inference:predict:640 - 
                        height: 192
                         width: 320
                  video_length: 257
                        prompt: ['Cartoon style. A man stands at a quiet bus stop on a sunny afternoon. Then, a bright yellow bus approaches.']
                    neg_prompt: ['Aerial view, aerial view, overexposed, low quality, deformation, a poor composition, bad hands, bad teeth, bad eyes, bad limbs, distortion']
                          seed: 5
                   infer_steps: 50
         num_videos_per_prompt: 1
                guidance_scale: 1.0
                      n_tokens: 15600
                    flow_shift: 7.0
       embedded_guidance_scale: 6.0
100%|██████████| 50/50 [02:04<00:00,  2.49s/it]
2025-01-31 15:17:32.074 | INFO     | hyvideo.vae.unet_causal_3d_blocks:forward:911 

In [ ]:
hunyuan_video_sampler.vae.adaptor=adaptor
args.infer_steps=50
args.video_length=128+1
height=720//3
width=1280//3
for seed in range(10):
    outputs = hunyuan_video_sampler.predict(
            prompt=prompt12, 
            height=height,
            width=width,
            video_length=args.video_length,
            seed=seed,
            negative_prompt=args.neg_prompt,
            infer_steps=args.infer_steps,
            guidance_scale=args.cfg_scale,
            num_videos_per_prompt=args.num_videos,
            flow_shift=args.flow_shift,
            batch_size=args.batch_size,
            embedded_guidance_scale=args.embedded_cfg_scale
        )
    samples = outputs['samples']
    save_path = f"output/raw_prompt1_{seed}.mp4"
    save_videos_grid(samples, save_path, fps=24)

2025-01-31 15:39:57.935 | INFO     | hyvideo.inference:predict:580 - Input (height, width, video_length) = (180, 320, 129)
2025-01-31 15:39:57.949 | DEBUG    | hyvideo.inference:predict:640 - 
                        height: 192
                         width: 320
                  video_length: 129
                        prompt: ['Realistic style. A man stands at a quiet bus stop on a sunny afternoon. Then, a bright yellow bus approaches.']
                    neg_prompt: ['Aerial view, aerial view, overexposed, low quality, deformation, a poor composition, bad hands, bad teeth, bad eyes, bad limbs, distortion']
                          seed: 0
                   infer_steps: 50
         num_videos_per_prompt: 1
                guidance_scale: 1.0
                      n_tokens: 7920
                    flow_shift: 7.0
       embedded_guidance_scale: 6.0
100%|██████████| 50/50 [00:54<00:00,  1.08s/it]
2025-01-31 15:40:56.254 | INFO     | hyvideo.vae.unet_causal_3d_blocks:forward:911

In [ ]:
args.infer_steps=50
outputs = hunyuan_video_sampler.predict(
        prompt=prompt2, 
        height=args.video_size[0]//2,
        width=args.video_size[1]//2,
        video_length=args.video_length,
        seed=args.seed,
        negative_prompt=args.neg_prompt,
        infer_steps=args.infer_steps,
        guidance_scale=args.cfg_scale,
        num_videos_per_prompt=args.num_videos,
        flow_shift=args.flow_shift,
        batch_size=args.batch_size,
        embedded_guidance_scale=args.embedded_cfg_scale
    )
samples = outputs['samples']
save_path = f"output/raw_prompt2.mp4"
save_videos_grid(samples, save_path, fps=24)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:


outputs = hunyuan_video_sampler.predict(
        prompt=prompt1, 
        height=args.video_size[0],
        width=args.video_size[1],
        video_length=args.video_length,
        seed=args.seed,
        negative_prompt=args.neg_prompt,
        infer_steps=args.infer_steps,
        guidance_scale=args.cfg_scale,
        num_videos_per_prompt=args.num_videos,
        flow_shift=args.flow_shift,
        batch_size=args.batch_size,
        embedded_guidance_scale=args.embedded_cfg_scale
    )
samples = outputs['samples']
for i, sample in enumerate(samples):
    samples = outputs['samples']
    save_path = f"output/raw_prompt11.mp4"
    save_videos_grid(sample, save_path, fps=24)

outputs = hunyuan_video_sampler.predict(
        prompt=prompt2, 
        height=args.video_size[0],
        width=args.video_size[1],
        video_length=args.video_length,
        seed=args.seed,
        negative_prompt=args.neg_prompt,
        infer_steps=args.infer_steps,
        guidance_scale=args.cfg_scale,
        num_videos_per_prompt=args.num_videos,
        flow_shift=args.flow_shift,
        batch_size=args.batch_size,
        embedded_guidance_scale=args.embedded_cfg_scale
    )
samples = outputs['samples']
for i, sample in enumerate(samples):
    samples = outputs['samples']
    save_path = f"output/raw_prompt22.mp4"
    save_videos_grid(sample, save_path, fps=24)